In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import ensemble
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, StratifiedKFold
from xgboost.sklearn import XGBClassifier
import xgboost as xgb
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix

In [52]:
diabetic7 = pd.read_csv("./dataset_diabetes/diabetic7.csv")

In [60]:
diabetic7 = diabetic7.drop(['admission_type_id'], axis=1)

In [61]:
le = preprocessing.LabelEncoder()

In [62]:
categorical = diabetic7.select_dtypes(include = ["object"]).columns
numerical = diabetic7.select_dtypes(exclude = ["object"]).columns
print("Number of Categorical features: " + str(len(categorical)))
print("Number of Numerical features: " + str(len(numerical)))
diabetic7_categorical = diabetic7[categorical]
diabetic7_numerical = diabetic7[numerical]

Number of Categorical features: 28
Number of Numerical features: 15


In [63]:
categorical= list(categorical)
numerical = list(numerical)

In [64]:
for i in categorical:
    diabetic7[i] = le.fit_transform(diabetic7[i])

In [65]:
answer = diabetic7['target']
diabetic7 = diabetic7.drop(['target'], axis=1)

In [23]:
import pickle

pickle_path = "./xgboost_bestauc_pickle"

with open(pickle_path, 'rb') as handle:
    mod = pickle.load(handle)

In [24]:
mod

XGBClassifier(base_score=0.5, booster='gbtree', class_weight={0: 1, 1: 8},
       colsample_bylevel=1, colsample_bytree=0.6, gamma=0.8,
       learning_rate=0.1, max_delta_step=0, max_depth=5,
       min_child_weight=7, min_samples_leaf=1, min_samples_split=4,
       missing=nan, n_estimators=280, n_jobs=1, njobs=-1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0.4,
       reg_lambda=1, scale_pos_weight=9, seed=None, silent=True,
       subsample=0.9)

In [81]:
holdout = pd.read_csv("./dataset_diabetes/end_holdout.csv")

In [73]:
holdout.shape

(14910, 42)

In [74]:
diabetic7.shape

(84432, 42)

In [59]:
ho = holdout.columns
di = diabetic7.columns
set(di) - set(ho)

set()

In [82]:
holdout = holdout.drop(['admission_type_id'], axis=1)

In [83]:
categorical = holdout.select_dtypes(include = ["object"]).columns
numerical = holdout.select_dtypes(exclude = ["object"]).columns
print("Number of Categorical features: " + str(len(categorical)))
print("Number of Numerical features: " + str(len(numerical)))
holdout_categorical = holdout[categorical]
holdout_numerical = holdout[numerical]

Number of Categorical features: 28
Number of Numerical features: 15


In [84]:
categorical= list(categorical)
numerical = list(numerical)

In [85]:
for i in categorical:
    holdout[i] = le.fit_transform(holdout[i])

In [86]:
answer = holdout['target']
holdout = holdout.drop(['target'], axis=1)

In [76]:
from sklearn.metrics import roc_auc_score, recall_score, precision_score, accuracy_score

In [88]:
print("auc holdout score: ", roc_auc_score(answer, mod.predict_proba(holdout)[:,1]))
print("recall holdout score: ",recall_score(answer, mod.predict(holdout)))
print("precision holdout score: ", precision_score(answer, mod.predict(holdout)))
print("accuracy holdout score: ",accuracy_score(answer, mod.predict(holdout)))
print(confusion_matrix(answer, mod.predict(holdout)))

auc holdout score:  0.6794116626285129
recall holdout score:  0.7157151265450264
precision holdout score:  0.1649037157580689
accuracy holdout score:  0.5545942320590208
[[7053 6158]
 [ 483 1216]]
